In [4]:
from parse_documents import *

# filepath = 'upload/Jean-Baptiste_Perronneau_-_Magdaleine_Pinceloup_de_la_Grange,_née_de_Parseval.jpg'  # Replace with the path of your file
# filepath = "upload/World_Energy_By_Country_And_Region_1965_to_2023.csv"
filepath = 'upload/Diabetic-retinopathy-identification-using-parallel-convo_2023_Expert-Systems.pdf'
parse_document(filepath)



<class 'dict'>
dict_keys(['metadata', 'content', 'status'])
{'pdf:PDFVersion': '1.7', 'xmp:CreatorTool': 'Elsevier', 'pdf:docinfo:title': 'Diabetic retinopathy identification using parallel convolutional neural network based feature extractor and ELM classifier', 'pdf:hasXFA': 'false', 'access_permission:can_print_degraded': 'true', 'X-TIKA:Parsed-By-Full-Set': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.pdf.PDFParser'], 'X-TIKA:content_handler': 'ToTextContentHandler', 'pdf:num3DAnnotations': '0', 'dc:format': 'application/pdf; version=1.7', 'pdf:docinfo:custom:robots': 'noindex', 'pdf:docinfo:creator_tool': 'Elsevier', 'access_permission:fill_in_form': 'true', 'pdf:docinfo:custom:CreationDate--Text': '7th February 2023', 'pdf:hasCollection': 'false', 'pdf:encrypted': 'false', 'dc:title': 'Diabetic retinopathy identification using parallel convolutional neural network based feature extractor and ELM classifier', 'pdf:containsNonEmbeddedFont': 'false', 'xmp:CreateD